In [1]:
import numpy as np
import tensorflow as tf

In [10]:
seq_length=512
batch_size=16
split = 0.9

In [3]:
with open('./npys/movies-xids.npy', 'rb') as f:
    xids = np.load(f, allow_pickle=True)

with open('./npys/movies-xmask.npy', 'rb') as f:
    xmask = np.load(f, allow_pickle=True)

with open('./npys/labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle=True)

In [4]:
dataset = tf.data.Dataset.from_tensor_slices((xids, xmask, labels))

In [5]:
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (5,)), types: (tf.int64, tf.int64, tf.float64)>

In [6]:
def map_func(input_ids, mask, labels):
    return {'input_ids':input_ids,
            'attention_mask':mask}, labels

In [7]:
dataset = dataset.map(map_func=map_func)

In [8]:
dataset = dataset.shuffle(10000).batch(batch_size=batch_size, drop_remainder=True)

In [9]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.float64)>

In [15]:
size=int((xids.shape[0]/batch_size)*split)

train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [17]:
tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

In [18]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int64, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))